In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY=os.getenv("API_KEY")
openAI_API_KEY=os.getenv("openAI_API_KEY")

## Parsing through openAI

In [13]:
import openai
import pdfplumber
import json
from openai import OpenAI

# STEP 1: Load PDF and extract text
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

# STEP 2: Define JSON schema and prompt
def build_prompt(resume_text):
    json_schema = {
        "name": "",
        "email": "",
        "phone": "",
        "country": "",
        "city": "",
        "summary": "",
        "skills": [
            {
                'specialized skill': "",
                'common skill': ""
                }
            ],
        "experience": [
            {
                "job_title": "",
                "company": "",
                "start_date": "",
                "end_date": "",
                "description": ""
            }
        ],
        "education": [
            {
                "degree": "",
                "institution": "",
                "start_year": "",
                "end_year": ""
            }
        ],
        "enrichment parameters": [
            {
                "Employment Pattern & Progression": "",
                "Company Type & Sector": "",
                "Education Quality & Ranking": "",
                "Skill Demand & Market Relevance": "",
                "Leadership Experience": "",
                "Budget & Project Management": "",
                "International Experience & Mobility": "",
                "Soft Skills from Sales Calls": "",
                "Personality & Behavioral Traits": "",
                "Future Career Goals (Sales-Inferred)": "",
                "Salary Expectations (Sales-Inferred)": "",
                "JD Enrichment with Implied Preferences": "",
                "Cultural Fit Indicators": ""
            }
        ]
    }

    prompt = f"""
You are an expert resume parser. Convert the resume text below into this JSON format. Fill in all the relevant fields. Leave the enrichment_parameters field empty.
The JSON schema is as follows:

{json.dumps(json_schema, indent=2)}

Resume:
\"\"\"
{resume_text}
\"\"\"
"""
    return prompt

# STEP 3: Call OpenAI API
def call_openai(prompt):
    api_key = openAI_API_KEY
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    #return response['choices'][0]['message']['content']
    return response.choices[0].message.content

# STEP 4: Main function
def main():
    pdf_path = "./sample_CVs/John_Doe_CV.pdf"  # Change if needed
    resume_text = extract_text_from_pdf(pdf_path)
    prompt = build_prompt(resume_text)
    parsed_cv = call_openai(prompt)

    print("Parsed CV:")
    print(parsed_cv)

    #Optionally save to file
    with open("./gpt_parsed_CVs/John_Doe_parsed_cv.json", "w") as f:
        f.write(parsed_cv)

if __name__ == "__main__":
    main()


Parsed CV:
{
  "name": "John Doe",
  "email": "johndoe@email.com",
  "phone": "+1-234-567-8901",
  "country": "",
  "city": "",
  "summary": "Seasoned finance executive with 18+ years of experience in financial strategy, risk management, and capital raising. Expertise in scaling startups and optimizing financial operations to drive profitability and growth.",
  "skills": [
    {
      "specialized skill": "Financial Strategy & Planning, Risk Management, Mergers & Acquisitions, Venture Capital & Fundraising, Budgeting & Forecasting",
      "common skill": ""
    }
  ],
  "experience": [
    {
      "job_title": "Chief Financial Officer",
      "company": "XYZ Tech Solutions",
      "start_date": "2018",
      "end_date": "Present",
      "description": "Spearheaded fundraising efforts, securing $50M in venture capital funding. Optimized financial structures, reducing operational costs by 30%. Led M&A initiatives, successfully acquiring and integrating 3 companies."
    },
    {
      "j

## DEEPSEEK

In [14]:
from openai import OpenAI
import json

# filepath: /Users/prayagsharma/Documents/ACP/scripts/deepseek_enrichment.py
def enrich_cv_with_deepseek(api_key, cv_data):
    """
    Enriches the CV data using the DeepSeek reasoning model to fill in empty enrichment parameters.

    Args:
        api_key (str): The API key for the DeepSeek API.
        cv_data (dict): The CV data to be enriched.

    Returns:
        dict: The enriched CV data with filled enrichment parameters.
    """
    # Initialize the OpenAI client with the DeepSeek API key
    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

    # Round 1: Send the initial prompt
    messages = [
        {"role": "user", "content": """
        Analyze the provided CV data and infer the following enrichment parameters:
        - Employment Pattern & Progression: Describe the career trajectory and progression.
        - Company Type & Sector: Identify the type and sector of companies worked for.
        - Education Quality & Ranking: Assess the quality and ranking of educational institutions.
        - Skill Demand & Market Relevance: Evaluate the relevance of skills in the current market.
        - Leadership Experience: Highlight leadership roles and responsibilities.
        - Budget & Project Management: Detail experience in managing budgets and projects.
        - International Experience & Mobility: Indicate international exposure and mobility.
        - Soft Skills from Sales Calls: Infer soft skills demonstrated in sales or communication.
        - Personality & Behavioral Traits: Deduce personality traits and behaviors.
        - Future Career Goals (Sales-Inferred): Predict future career aspirations based on sales roles.
        - Salary Expectations (Sales-Inferred): Estimate salary expectations based on experience.
        - JD Enrichment with Implied Preferences: Enrich job descriptions with implied preferences.
        - Cultural Fit Indicators: Suggest cultural fit indicators for potential roles.
        """}
    ]
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )

    # Extract the assistant's response
    assistant_response = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_response})

    # Round 2: Send the CV data
    messages.append({"role": "user", "content": f"Here is the CV data: {json.dumps(cv_data)}. Please analyze and fill in the enrichment parameters. return the enriched CV data in JSON format."})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
        'type': 'json_object'
    }
    )

    # Extract the enriched CV data from the assistant's response
    enriched_cv_content = response.choices[0].message.content
    enriched_cv = json.loads(enriched_cv_content)  # Assuming the response contains the enriched CV data in JSON format
    return enriched_cv


# Example usage
if __name__ == "__main__":
    # Load the CV data from cv_0.json
    with open("/Users/prayagsharma/Documents/ACP/Affinda_parsed_CVs/cv_0.json", "r") as file:
        cv_data = json.load(file)

    # DeepSeek API key
    api_key = API_KEY

    # Enrich the CV data
    try:
        enriched_cv = enrich_cv_with_deepseek(api_key, cv_data)
        # Merge the enriched CV data into the original CV data
        cv_data['enrichment parameters'] = enriched_cv['enrichment parameters']
        # Save the updated cv_data to a new file
        with open("/Users/prayagsharma/Documents/ACP/gpt_enriched_CVs/cv_0_updated.json", "w") as newfile:
            json.dump(cv_data, newfile, indent=4)
        # Save the enriched CV data back to the file
        with open("/Users/prayagsharma/Documents/ACP/gpt_enriched_CVs/cv_0_enriched_deepseek.json", "w") as file:
            json.dump(enriched_cv, file, indent=4)
        print("CV enrichment completed successfully.")
    except Exception as e:
        print(f"Error during CV enrichment: {e}")

CV enrichment completed successfully.


## OPENAI

In [15]:
from openai import OpenAI
import json

# filepath: /Users/prayagsharma/Documents/ACP/scripts/openai_enrichment.py
def enrich_cv_with_openai(api_key, cv_data):
    """
    Enriches the CV data using the OpenAI API to fill in empty enrichment parameters.

    Args:
        api_key (str): The API key for the OpenAI API.
        cv_data (dict): The CV data to be enriched.

    Returns:
        dict: The enriched CV data with filled enrichment parameters.
    """
    # Initialize the OpenAI client
    client = OpenAI(api_key=api_key)

    # Round 1: Send the initial prompt
    messages = [
        {"role": "user", "content": """
        Analyze the provided CV data and infer the following enrichment parameters:
        - Employment Pattern & Progression: Describe the career trajectory and progression.
        - Company Type & Sector: Identify the type and sector of companies worked for.
        - Education Quality & Ranking: Assess the quality and ranking of educational institutions.
        - Skill Demand & Market Relevance: Evaluate the relevance of skills in the current market.
        - Leadership Experience: Highlight leadership roles and responsibilities.
        - Budget & Project Management: Detail experience in managing budgets and projects.
        - International Experience & Mobility: Indicate international exposure and mobility.
        - Soft Skills from Sales Calls: Infer soft skills demonstrated in sales or communication.
        - Personality & Behavioral Traits: Deduce personality traits and behaviors.
        - Future Career Goals (Sales-Inferred): Predict future career aspirations based on sales roles.
        - Salary Expectations (Sales-Inferred): Estimate salary expectations based on experience.
        - JD Enrichment with Implied Preferences: Enrich job descriptions with implied preferences.
        - Cultural Fit Indicators: Suggest cultural fit indicators for potential roles.
        """}
    ]
    response = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=messages
    )

    # Extract the assistant's response
    assistant_response = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_response})

    # Round 2: Send the CV data
    messages.append({"role": "user", "content": f"Here is the CV data: {json.dumps(cv_data)}. Please analyze and fill in the enrichment parameters. Return the enriched CV data in JSON format."})
    response = client.chat.completions.create(
        model="o3-mini-2025-01-31",
        messages=messages,
        response_format={"type": "json_object"}
    )

    # Extract the enriched CV data from the assistant's response
    enriched_cv_content = response.choices[0].message.content
    enriched_cv = json.loads(enriched_cv_content)  # Assuming the response contains the enriched CV data in JSON format
    return enriched_cv


# Example usage
if __name__ == "__main__":
    # Load the CV data from cv_0.json
    with open("/Users/prayagsharma/Documents/ACP/Affinda_parsed_CVs/cv_0.json", "r") as file:
        cv_data = json.load(file)

    # OpenAI API key
    api_key = openAI_API_KEY

    # Enrich the CV data
    try:
        enriched_cv = enrich_cv_with_openai(api_key, cv_data)
        # Save the enriched CV data back to the file
        with open("/Users/prayagsharma/Documents/ACP/gpt_enriched_CVs/cv_0_enriched_openAI.json", "w") as file:
            json.dump(enriched_cv, file, indent=4)
        print("CV enrichment completed successfully.")
    except Exception as e:
        print(f"Error during CV enrichment: {e}")

CV enrichment completed successfully.
